<a href="https://colab.research.google.com/github/dorian-goueytes/L1_P-M1_MIASHS/blob/main/TD2_HH_synapses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Installation des dépendances
!pip install brian2
!pip install brian2tools
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
import brian2tools as b2t

In [ ]:
# @title Fonctions pour injecter du courant et pour monitorer l'état de nos neurones
def plot_data(ref, state_monitor, title=None):
    """Plots the state_monitor variables ["vm", "I_e", "m", "n", "h"] vs. time.

    Args:
        state_monitor (StateMonitor): the data to plot
        title (string, optional): plot title to display
    """

    fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize = (12,7))
    ax1.plot(ref.t / b2.ms, ref.vm[0] / b2.mV, lw=2, color = 'b', linestyle = '-', alpha = 0.7)
    #plt.plot(state_monitor.t / b2.ms, state_monitor.vm[0] / b2.mV, lw=2, alpha = 0.7)
    ax1.set_xlabel("Temps [ms]")
    ax1.set_ylabel("Potentiel de Membrane [mV]")
    ax1.set_title("Neurone pré-synaptique")
    ax1.axhline(15, color = 'k', linestyle = '--')
    #ax1.set_ylim(-20,110)
    plt.grid()

    ax3.plot(state_monitor.t / b2.ms, state_monitor.vm[0] / b2.mV, lw=2, color = 'r', alpha = 0.7)
    ax3.set_xlabel("Temps [ms]")
    ax3.set_ylabel("Potentiel de Membrane [mV]")
    ax3.set_title("Neurone post-synaptique")
    ax3.axhline(15, color = 'k', linestyle = '--')
    #ax3.set_ylim(-20,110)
    plt.grid()

    ax2.plot(ref.t / b2.ms, ref.I_e[0] / b2.uamp, lw=2, color = 'b', linestyle = '-', alpha = 0.7)
    ax2.set_xlabel("Temps [ms]")
    ax2.set_ylabel("Courant Injecté [uA]")
    ax2.set_title("Stimulation des neurones présynaptiques")


    plt.tight_layout()
    plt.show()

def get_step_current(t_start, t_end, unit_time, amplitude, append_zero=True):

    """Creates a step current. If t_start == t_end, then a single
    entry in the values array is set to amplitude.

    Args:
        t_start (int): start of the step
        t_end (int): end of the step
        unit_time (Brian2 unit): unit of t_start and t_end. e.g. 0.1*brian2.ms
        amplitude (Quantity): amplitude of the step. e.g. 3.5*brian2.uamp
        append_zero (bool, optional): if true, 0Amp is appended at t_end+1.
        Without that trailing 0, Brian reads out the last value in the array (=amplitude) for all indices > t_end.

    Returns:
        TimedArray: Brian2.TimedArray
    """

    assert isinstance(t_start, int), "t_start_ms must be of type int"
    assert isinstance(t_end, int), "t_end must be of type int"
    assert b2.units.fundamentalunits.have_same_dimensions(amplitude, b2.amp), \
        "amplitude must have the dimension of current e.g. brian2.uamp"
    tmp_size = 1 + t_end  # +1 for t=0
    if append_zero:
        tmp_size += 1
    tmp = np.zeros((tmp_size, 1)) * b2.amp
    tmp[t_start: t_end + 1, 0] = amplitude
    curr = b2.TimedArray(tmp, dt=1. * unit_time)
    return curr

In [ ]:
# @title Définition du modèle de Hogkin et Huxley
def simulate_HH_neuron(input_current, simulation_time, kcurrent = None, N_pre = 1, N_post = 1, N_inib = 0, snp = True, weight = 0.05):

    """A Hodgkin-Huxley neuron implemented in Brian2.

    Args:
        input_current (TimedArray): Input current injected into the HH neuron
        simulation_time (float): Simulation time [seconds]

    Returns:
        StateMonitor: Brian2 StateMonitor with recorded fields
        ["vm", "I_e", "m", "n", "h"]
    """

    # neuron parameters
    El = 10.6 * b2.mV
    if kcurrent == None:
      EK = -12 * b2.mV
    else:
      EK = kcurrent* b2.mV
    ENa = 115 * b2.mV
    gl = 0.3 * b2.msiemens
    gK = 36 * b2.msiemens
    gNa = 120 * b2.msiemens

    C = 1 * b2.ufarad

    # forming HH model with differential equations
    eqs = """
    I_e = input_current(t,i) : amp
    membrane_Im = I_e + gNa*m**3*h*(ENa-vm) + gl*(El-vm) + gK*n**4*(EK-vm) : amp
    alphah = .07*exp(-.05*vm/mV)/ms    : Hz
    alpham = .1*(25*mV-vm)/(exp(2.5-.1*vm/mV)-1)/mV/ms : Hz
    alphan = .01*(10*mV-vm)/(exp(1-.1*vm/mV)-1)/mV/ms : Hz
    betah = 1./(1+exp(3.-.1*vm/mV))/ms : Hz
    betam = 4*exp(-.0556*vm/mV)/ms : Hz
    betan = .125*exp(-.0125*vm/mV)/ms : Hz
    dh/dt = alphah*(1-h)-betah*h : 1
    dm/dt = alpham*(1-m)-betam*m : 1
    dn/dt = alphan*(1-n)-betan*n : 1
    dvm/dt = membrane_Im/C : volt
    """

        # forming HH model with differential equations
    eqs2 = """
    membrane_Im = gNa*m**3*h*(ENa-vm) + gl*(El-vm) + gK*n**4*(EK-vm) : amp
    alphah = .07*exp(-.05*vm/mV)/ms    : Hz
    alpham = .1*(25*mV-vm)/(exp(2.5-.1*vm/mV)-1)/mV/ms : Hz
    alphan = .01*(10*mV-vm)/(exp(1-.1*vm/mV)-1)/mV/ms : Hz
    betah = 1./(1+exp(3.-.1*vm/mV))/ms : Hz
    betam = 4*exp(-.0556*vm/mV)/ms : Hz
    betan = .125*exp(-.0125*vm/mV)/ms : Hz
    dh/dt = alphah*(1-h)-betah*h : 1
    dm/dt = alpham*(1-m)-betam*m : 1
    dn/dt = alphan*(1-n)-betan*n : 1
    dvm/dt = membrane_Im/C : volt
    """


    ## Presynaptic neurons
    neuron_pre = b2.NeuronGroup(N_pre, eqs, threshold='vm > 15*mV', refractory= 'vm > 0*mV', method="exponential_euler")
    # parameter initialization
    neuron_pre.vm = 0
    neuron_pre.m = 0.05
    neuron_pre.h = 0.60
    neuron_pre.n = 0.32
    ## Postsynaptic neurons
    neuron_post = b2.NeuronGroup(N_post, eqs2, threshold='vm > 15*mV', refractory= 'vm > 0*mV', method="exponential_euler")
    # parameter initialization
    neuron_post.vm = 0
    neuron_post.m = 0.05
    neuron_post.h = 0.60
    neuron_post.n = 0.32

    #synapses
    if snp ==True:
      syn=b2.Synapses(neuron_pre,neuron_post,model="w:1",on_pre="m+=w", delay=5*b2.ms, multisynaptic_index = 'test')
      #for s in range(0, N_pre):
      syn.connect(i = range(0, N_pre), j = 0)
      syn.w =  weight*N_pre

    # Synapses inhibitrices
    if N_inib !=0 and snp ==True:
      neuron_inhib = b2.NeuronGroup(N_inib, eqs, threshold='vm > 15*mV', refractory= 'vm > 0*mV', method="exponential_euler")
      neuron_inhib.vm = 0
      neuron_inhib.m = 0.05
      neuron_inhib.h = 0.60
      neuron_inhib.n = 0.32
      syn_inh=b2.Synapses(neuron_inhib,neuron_post,model="w:1",on_pre="m+=w", delay=5*b2.ms, multisynaptic_index = 'test')
      syn_inh.connect(i = range(0, N_inib), j = 0)
      syn_inh.w =  -weight*N_inib



    # tracking parameters
    st_mon_pre = b2.StateMonitor(neuron_pre, ["vm", "I_e",'m'], record=True)
    st_mon_post = b2.StateMonitor(neuron_post, ["vm"], record=True)

    # running the simulation
    if snp == False:
      hh_net = b2.Network(neuron_pre, neuron_post, st_mon_pre, st_mon_post)
    if snp ==True:
      hh_net = b2.Network(neuron_pre, neuron_post,syn, st_mon_pre, st_mon_post)
    if N_inib !=0 and snp ==True:
      hh_net.add(neuron_inhib)
      hh_net.add(syn_inh)
    #hh_net.add(st_mon)
    hh_net.run(simulation_time)


    return st_mon_pre, st_mon_post

### Création de deux neurones de Hodgkin et Huxley
Nous allons créer deux neurones et stimuler le premier

**Q01: Suite à la stimulation qu'observez-vous pour l'activité des deux neurones? Pourquoi le neurone post-synaptique n'émet pas de potentiel d'action?**

**Q02 : Interprétez ce qu'il se passe lorsque vous cochez la case 'synapse'**

In [ ]:
# @title Pré-requis
current = get_step_current(10, 11, b2.ms, 7.6 * b2.uA)
synapse = False #@param {type: "boolean"}
pre, post= simulate_HH_neuron(current, 70 * b2.ms,-12,3,1,0, synapse,weight = 0.01)
plot_data(pre,post, title="HH Neuron, step current")

### Poids de la synapse
Nous avons vu que malgré la présence d'une synapse un potentiel d'action au niveau du neurone pré-synaptique ne déclenche pas de PA au niveau post-synaptique.

**Q03 : Pouvez-vous proposer une hypothèse expliquant cette observation**

**Q04 : Que se passe-t-il lorsque vous manipulez la variable "poid_synaptique"? Quel phénomène observez-vous en l'absence de PA?**


In [ ]:
# @title Notion de poids Synaptique
poids_synaptique = 0.01 # @param {type:"slider", min:0, max:0.2, step:0.01}
pre, post= simulate_HH_neuron(current, 70 * b2.ms,-12,1,1,0, True, poids_synaptique)
plot_data(pre,post, title="HH Neuron, step current")

### Notion d'intégration synaptique
Une des propriété principale d'un neurone est d'intégrer l'information de nombreuse sources différentes. Cela se matérialise par la capacité à recevoir et intégrer le signal de nombreuses synapses

La variable neurones_presyn correspond au nombre de neurones faisant synapse avec notre neurone post-synaptique

**Q05 : Pour une valeur faible de poids_synaptique, augmentez progressivement le nombre de neurons pré-synaptiques. Qu'observez-vous et pourquoi?**

**Q06 : Quelle est la nature de l'interaction entre le poids synaptique et le nombre de neurone pré-synaptiques?**

In [ ]:
# @title Variation du nombre de synapses
poids_synaptique = 0.01 # @param {type:"slider", min:0, max:0.2, step:0.01}
neurones_presyn = 4 # @param {type:"slider", min:1, max:20, step:1}
pre, post= simulate_HH_neuron(current, 70 * b2.ms,-12,neurones_presyn,1,0, True, poids_synaptique)
plot_data(pre,post, title="HH Neuron, step current")

### Neurone inhibiteur
Une synapse ne donne pas nécessairement lieu à un potentiel post-synaptique excitateur (PPSE), mais peut aussi donner lieu à un potentiel post-synaptique inhibiteur (PPSI)

Ici nous allons remplacer nos synapses excitatrice par des synapses inhibitrice

**Q07 :Qu'observez-vous au niveau du potentiel membrainaire post-synaptique lorsque vous modulez le nombre de synapses inhibitrices? Quel est le nom de ce phénomène**

**Q08 : Est-il possible de voir émerger un potentiel d'action dans ces conditions? Pourquoi?**

In [ ]:
# @title Variation du nombre de synapses
synapse_exc = 1 # @param {type:"slider", min:1, max:2, step:1}
synapse_inh = 7 # @param {type:"slider", min:3, max:15, step:1}
pre, post= simulate_HH_neuron(current, 70 * b2.ms,-12,synapse_exc,1,synapse_inh, True,)
plot_data(pre,post, title="HH Neuron, step current")

### Equilibre synaptique
Nous allons maintenant manipuler l'équilibre entre le nombre de synapses inhibitrices et excitatrices.

Notre objectif est de rétablir l'émergence d'un potentiel d'action

**Q09 : Pour quel réglage des paramètre voyez vous re-émerger un potentiel d'action?**

**Q10 : Comment interprétez vous ce résultat?**

In [ ]:
# @title Variation du nombre de synapses
synapse_exc = 1 # @param {type:"slider", min:1, max:15, step:1}
synapse_inh = 1 # @param {type:"slider", min:1, max:15, step:1}
pre, post= simulate_HH_neuron(current, 70 * b2.ms,-12,synapse_exc,1,synapse_inh, True,)
plot_data(pre,post, title="HH Neuron, step current")